In [20]:
import pandas as pd

from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [21]:
F_selection = pd.read_csv("F_Selection/top_N_features_SelectFromModel_XGBAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [22]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [23]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_9520\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [24]:
top = top_10

rank_data_10 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.15493434647052
R-squared (XGBoost): -1.595498388456678


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,72.996918
13894,Dikembe Mutombo,1,C,39.0,72.996872
13740,Hakeem Olajuwon,-1,C,0.0,72.996506
13601,Kevin Garnett,11T,PF,1.0,0.821744
13931,Eddie Jones,-1.0,SG,0.0,0.795795
13804,Tim Hardaway,-1.0,PG,0.0,0.757510
13889,Tim Duncan,5T,PF,4.0,0.571037
13948,Grant Hill,-1.0,SF,0.0,0.398445
13605,David Robinson,3,C,10.0,0.254776
13956,Charles Oakley,-1,PF,0.0,0.250847


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.43084755235493
R-squared (XGBoost): 0.32702298191098456


,Player_name,Rank,Pos,Points_won,Predicted_points
13424,Gary Payton,3,PG,6.0,72.978668
13269,Alonzo Mourning,1,C,89.0,72.966515
13182,Dikembe Mutombo,2,C,10.0,22.986021
13039,Hakeem Olajuwon,7T,C,1.0,22.982710
13438,Jason Kidd,-1.0,PG,0.0,0.681837
13077,John Stockton,-1.0,PG,0.0,0.487725
13286,Kurt Thomas,-1.0,PF,0.0,0.468560
13368,Jim McIlvaine,-1.0,C,0.0,0.467579
13332,Ron Harper,-1.0,SG,0.0,0.463327
13184,Andrew DeClercq,-1.0,C,0.0,0.346412


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.33695684271592
R-squared (XGBoost): -0.6849216917087653


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,218.171967
1184,Nikola Jokić,-1.0,C,0.0,103.051460
1026,Myles Turner,-1.0,C,0.0,86.322433
1212,Jayson Tatum,-1.0,SF,0.0,37.950500
1111,Giannis Antetokounmpo,6.0,PF,58.0,12.829240
933,Al Horford,9,C,3.0,7.330245
640,Pascal Siakam,-1.0,PF,0.0,5.816088
1007,Joel Embiid,-1,C,0.0,5.778539
824,De'Anthony Melton,-1.0,SG,0.0,4.128823
942,Fred VanVleet,-1.0,PG,0.0,4.038467


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 131.38222190374
R-squared (XGBoost): 0.6912410729753145


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,233.187454
323,Brook Lopez,2.0,C,309.0,145.719894
206,Nic Claxton,9T,C,7.0,135.382263
357,Nikola Jokić,-1.0,C,0.0,29.756277
75,Myles Turner,-1.0,C,0.0,5.720732
208,Walker Kessler,-1.0,C,0.0,4.441282
137,Evan Mobley,3.0,PF,101.0,4.317959
104,Jimmy Butler,12,PF,1.0,4.151076
164,Mitchell Robinson,-1.0,C,0.0,3.231432
392,Jacob Gilyard,-1.0,PG,0.0,2.934465


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,3,5T,-1,1,1,2,1,8,-1,...,1.0,2,1,-1.0,3,2.0,1.0,1.0,3.0,1.0
1,1,1,3T,-1,-1,-1,1,5,10T,1,...,7.0,4,3,-1.0,15T,1.0,11,9.0,-1.0,2.0
2,-1,2,-1,-1,3T,11,-1,-1,1,3,...,-1,3.0,2,1,-1.0,-1,3.0,3,-1.0,9T
3,11T,7T,1,1,-1,3,-1,9,7,8,...,-1.0,8,5.0,-1,1.0,-1,2,-1,-1.0,-1.0
4,-1.0,-1.0,-1,1,-1,-1.0,7,-1,-1,-1.0,...,4.0,1,-1.0,-1.0,6,-1.0,10.0,-1,6.0,-1.0
5,-1.0,-1.0,-1.0,-1.0,-1,4,6,-1,3,-1.0,...,8T,15T,-1.0,-1.0,14.0,5.0,8T,5.0,9,-1.0
6,5T,-1.0,8T,-1.0,-1.0,2,5,-1,-1.0,-1,...,-1.0,5.0,10,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,3.0
7,-1.0,-1.0,5T,5T,-1,-1,-1,-1,-1,20.0,...,-1.0,15T,-1.0,3,8.0,-1.0,-1.0,-1.0,-1,12
8,3,-1.0,8T,11T,-1.0,11,10T,4,4,16T,...,-1.0,-1.0,-1,-1,-1.0,-1.0,8T,-1.0,-1.0,-1.0
9,-1,-1.0,-1.0,11T,-1,-1.0,-1.0,-1.0,4,11T,...,-1.0,-1.0,-1.0,-1,-1.0,-1.0,8T,-1.0,-1.0,-1.0


In [25]:
top = top_20

rank_data_20 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.0663944525245
R-squared (XGBoost): -1.581273313496765


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.998245
13764,Gary Payton,2,PG,37.0,72.998131
13894,Dikembe Mutombo,1,C,39.0,72.998131
13605,David Robinson,3,C,10.0,1.493343
13836,Michael Jordan,4,SG,6.0,1.114389
13637,Dennis Rodman,5T,PF,4.0,1.013601
13889,Tim Duncan,5T,PF,4.0,0.960586
13628,Zydrunas Ilgauskas,-1.0,C,0.0,0.906397
13956,Charles Oakley,-1,PF,0.0,0.605369
13744,Bo Outlaw,5T,PF,4.0,0.382167


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 11.408020159445476
R-squared (XGBoost): 0.32836691644782345


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,72.971352
13424,Gary Payton,3,PG,6.0,72.967888
13182,Dikembe Mutombo,2,C,10.0,22.977417
13039,Hakeem Olajuwon,7T,C,1.0,22.976364
13488,Tim Duncan,5T,PF,2.0,2.873286
13434,David Robinson,4,C,3.0,2.796460
13184,Andrew DeClercq,-1.0,C,0.0,0.366660
13183,Andrew DeClercq,-1.0,C,0.0,0.291504
13286,Kurt Thomas,-1.0,PF,0.0,0.291426
13332,Ron Harper,-1.0,SG,0.0,0.286797


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 14.649247321954887
R-squared (XGBoost): -0.5108587736138597


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,203.731369
1184,Nikola Jokić,-1.0,C,0.0,117.710701
1026,Myles Turner,-1.0,C,0.0,54.020695
1176,Jae Crowder,-1,PF,0.0,18.338499
1094,Jose Alvarado,-1.0,PG,0.0,13.203436
1111,Giannis Antetokounmpo,6.0,PF,58.0,10.556326
1007,Joel Embiid,-1,C,0.0,10.154263
997,Jaren Jackson Jr.,5.0,PF,99.0,8.204347
933,Al Horford,9,C,3.0,7.280368
1287,Delon Wright,-1.0,SG,0.0,7.251378


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 182.73148253709942
R-squared (XGBoost): 0.5705661263430133


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,212.321701
137,Evan Mobley,3.0,PF,101.0,68.239845
206,Nic Claxton,9T,C,7.0,34.737213
323,Brook Lopez,2.0,C,309.0,33.099945
176,Giannis Antetokounmpo,6.0,PF,14.0,15.496645
392,Jacob Gilyard,-1.0,PG,0.0,7.914303
436,Draymond Green,4,PF,34.0,4.975989
477,Bruno Fernando,-1.0,C,0.0,3.015898
357,Nikola Jokić,-1.0,C,0.0,2.275997
530,Bobby Portis,-1.0,PF,0.0,2.119275


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,5T,-1,1,1,2,1,8,-1,...,1.0,2,3,-1.0,15T,2.0,1.0,1.0,3.0,1.0
1,2,3,3T,1,-1,-1,1,-1,4,8,...,7.0,1,5.0,-1.0,-1.0,1.0,3.0,9.0,-1.0,3.0
2,1,2,1,1,3T,11,-1,9,4,-1.0,...,-1.0,10T,1,2.0,3,-1,-1.0,-1,-1.0,9T
3,3,7T,-1,-1,-1,4,-1,4,7,-1.0,...,-1.0,4,2,1,-1.0,-1.0,-1.0,5.0,-1,2.0
4,4,5T,8T,-1,-1,3,6,5,10T,1,...,-1.0,3.0,-1.0,5T,1.0,-1,11,-1,-1.0,6.0
5,5T,4,-1.0,5T,-1.0,-1.0,-1,18T,3,3,...,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,10.0,3,6.0,-1.0
6,5T,-1.0,7,-1.0,-1,-1.0,7,-1,-1,13T,...,-1.0,-1.0,-1.0,-1,-1,-1,2,-1.0,-1,4
7,-1.0,-1.0,2,11T,-1.0,12,5,3,1,-1,...,-1.0,-1.0,-1.0,-1.0,8.0,6T,8T,-1.0,5.0,-1.0
8,-1,-1.0,-1.0,11T,-1.0,-1.0,-1.0,-1.0,-1,4,...,-1,-1.0,-1.0,-1.0,14.0,-1,6T,6.0,9,-1.0
9,5T,-1.0,-1.0,3,-1,11,-1.0,10,-1.0,6,...,8T,-1.0,4.0,-1.0,-1,-1.0,-1.0,11T,-1.0,-1.0


In [26]:
top = top_30

rank_data_30 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.00124724624659
R-squared (XGBoost): -1.5708065752680382


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.998070
13894,Dikembe Mutombo,1,C,39.0,72.998070
13764,Gary Payton,2,PG,37.0,72.998001
13605,David Robinson,3,C,10.0,8.255016
13836,Michael Jordan,4,SG,6.0,7.124970
13505,Alonzo Mourning,-1,C,0.0,2.999583
13954,Rick Fox,-1,SF,0.0,2.999583
13626,John Starks,-1,SG,0.0,2.999583
13631,Derrick McKey,-1,SF,0.0,2.999583
13675,Nate McMillan,-1,SG,0.0,2.999583


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 12.809341898785906
R-squared (XGBoost): 0.24586583144906937


,Player_name,Rank,Pos,Points_won,Predicted_points
13424,Gary Payton,3,PG,6.0,72.988205
13269,Alonzo Mourning,1,C,89.0,72.985962
13039,Hakeem Olajuwon,7T,C,1.0,22.996109
13182,Dikembe Mutombo,2,C,10.0,22.987280
13434,David Robinson,4,C,3.0,18.359119
13450,Theo Ratliff,5T,PF,2.0,12.591175
13165,Patrick Ewing,-1,C,0.0,12.572731
13488,Tim Duncan,5T,PF,2.0,4.591444
13380,Mookie Blaylock,-1,PG,0.0,4.578422
13121,Bo Outlaw,-1,PF,0.0,4.570129


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 14.95325515166594
R-squared (XGBoost): -0.5422128006619171


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,126.410309
1184,Nikola Jokić,-1.0,C,0.0,74.218285
1026,Myles Turner,-1.0,C,0.0,47.904697
1046,Sekou Doumbouya,-1.0,PF,0.0,24.511822
1275,Stephen Curry,-1.0,PG,0.0,21.034998
762,Jaylen Brown,-1.0,SF,0.0,20.838024
1007,Joel Embiid,-1,C,0.0,20.803072
992,Paul George,-1.0,PF,0.0,20.268303
813,Ricky Rubio,-1.0,PG,0.0,19.549118
713,Marcus Smart,1,PG,257.0,19.187748


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 339.6986331925528
R-squared (XGBoost): 0.20168053198909164


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,86.675583
76,Mac McClung,-1.0,SG,0.0,71.112129
206,Nic Claxton,9T,C,7.0,17.720501
59,Jaren Jackson Jr.,1.0,C,391.0,16.670767
208,Walker Kessler,-1.0,C,0.0,11.106157
524,Bam Adebayo,5.0,C,18.0,9.763640
197,Kawhi Leonard,-1,SF,0.0,8.470324
436,Draymond Green,4,PF,34.0,6.001590
176,Giannis Antetokounmpo,6.0,PF,14.0,5.919380
497,Joel Embiid,9T,C,7.0,4.689121


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,3,5T,-1,1,-1,2,1,4,-1,...,1.0,1,3,-1.0,15T,2.0,1.0,1.0,3.0,2.0
1,1,1,3T,1,-1,1,1,4,4,3,...,7.0,2,5.0,-1.0,6,1.0,10.0,9.0,-1.0,-1.0
2,2,7T,-1,1,3T,11,-1,9,8,8,...,4.0,8,1,2.0,1.0,-1,-1.0,-1,-1.0,9T
3,3,2,1,-1,-1,4,-1,-1,1,4,...,5,10T,2,1,8.0,-1,-1.0,5.0,-1.0,1.0
4,4,4,8T,-1,-1,3,6,5,7,1,...,13T,6.0,-1.0,3,-1,-1.0,11,-1,-1.0,-1.0
5,-1,5T,-1,3,-1,-1,7,18T,13T,24T,...,-1.0,13T,11T,-1,-1.0,-1,8T,-1,-1.0,5.0
6,-1,-1,3T,2,-1,5.0,-1,6.0,-1,5,...,-1,4,-1.0,-1,3,-1.0,-1,3,-1,-1
7,-1,5T,-1,5T,3T,8,10T,-1,10T,13T,...,8T,-1,4.0,-1,-1.0,5.0,-1.0,-1.0,-1.0,4
8,-1,-1,7,-1,-1,13T,4,-1.0,6,11T,...,-1.0,15T,-1,-1,-1.0,6T,2,-1.0,-1.0,6.0
9,-1,-1,5T,-1,-1,13T,-1,8,13T,2,...,-1.0,3.0,10,-1.0,-1,3.0,-1.0,11T,1,9T


In [27]:
top = top_40

rank_data_40 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.00124724624659
R-squared (XGBoost): -1.5708065752680382


,Player_name,Rank,Pos,Points_won,Predicted_points
13740,Hakeem Olajuwon,-1,C,0.0,72.998070
13894,Dikembe Mutombo,1,C,39.0,72.998070
13764,Gary Payton,2,PG,37.0,72.998001
13605,David Robinson,3,C,10.0,8.255016
13836,Michael Jordan,4,SG,6.0,7.124970
13505,Alonzo Mourning,-1,C,0.0,2.999583
13954,Rick Fox,-1,SF,0.0,2.999583
13626,John Starks,-1,SG,0.0,2.999583
13631,Derrick McKey,-1,SF,0.0,2.999583
13675,Nate McMillan,-1,SG,0.0,2.999583


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 12.547640914665665
R-squared (XGBoost): 0.2612731533574013


,Player_name,Rank,Pos,Points_won,Predicted_points
13269,Alonzo Mourning,1,C,89.0,14.522191
13434,David Robinson,4,C,3.0,13.396243
13450,Theo Ratliff,5T,PF,2.0,7.838060
13424,Gary Payton,3,PG,6.0,7.743646
13165,Patrick Ewing,-1,C,0.0,7.453379
13370,Shawn Kemp,-1,PF,0.0,7.333355
13182,Dikembe Mutombo,2,C,10.0,6.177183
13488,Tim Duncan,5T,PF,2.0,5.644080
13271,Latrell Sprewell,-1,SF,0.0,4.605700
13313,Brevin Knight,-1,PG,0.0,4.540176


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 5.91697545580952
R-squared (XGBoost): 0.3897492421150066


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,150.493347
1026,Myles Turner,-1.0,C,0.0,52.287209
813,Ricky Rubio,-1.0,PG,0.0,36.319477
610,Michael Porter Jr.,-1.0,SF,0.0,32.782417
1046,Sekou Doumbouya,-1.0,PF,0.0,32.078892
1007,Joel Embiid,-1,C,0.0,25.253502
762,Jaylen Brown,-1.0,SF,0.0,20.240145
1275,Stephen Curry,-1.0,PG,0.0,19.835495
992,Paul George,-1.0,PF,0.0,15.744511
1111,Giannis Antetokounmpo,6.0,PF,58.0,7.875853


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 330.77613420593605
R-squared (XGBoost): 0.22264913164867928


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,80.479675
76,Mac McClung,-1.0,SG,0.0,75.434998
59,Jaren Jackson Jr.,1.0,C,391.0,27.905066
206,Nic Claxton,9T,C,7.0,12.612930
158,Jrue Holiday,7T,PG,8.0,8.156627
176,Giannis Antetokounmpo,6.0,PF,14.0,5.477000
403,Zion Williamson,-1.0,PF,0.0,5.373319
197,Kawhi Leonard,-1,SF,0.0,4.599631
301,Anthony Davis,-1,C,0.0,3.802031
75,Myles Turner,-1.0,C,0.0,3.022129


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,-1,1,1,1,1,-1,2,1,4,-1,...,1.0,4,3,-1.0,3,2.0,1.0,1.0,3.0,2.0
1,1,4,-1,-1,-1,1,1,-1,4,3,...,7.0,-1.0,5.0,-1.0,6,-1.0,2,5.0,-1.0,-1.0
2,2,5T,7,-1,3T,-1,-1,4,8,1,...,4.0,15T,1,2.0,15T,1.0,10.0,-1,-1.0,1.0
3,3,3,3T,3,-1,4,7,5,7,4,...,5,-1,2,1,-1,-1,3.0,-1,-1.0,9T
4,4,-1,-1,1,3T,13T,-1,9,1,5,...,13T,13T,-1.0,3,14.0,-1,-1,9.0,-1.0,7T
5,-1,-1,5T,8T,-1,11,6,-1.0,-1,20.0,...,13T,1,-1.0,-1,1.0,5.0,11,3,-1,6.0
6,-1,2,8T,11T,-1,3,10T,18T,13T,6,...,8T,10T,-1,4.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0
7,-1,5T,3T,2,-1,5.0,-1,-1,10T,11T,...,8T,2,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1
8,-1,-1,8T,-1,-1,13T,4,-1.0,6,24T,...,6,8,11T,-1,-1.0,3.0,-1.0,-1.0,-1.0,-1
9,-1,-1,-1,5T,-1,8,-1,-1,3,2,...,-1.0,13T,13T,5T,4.0,6T,-1.0,-1,6.0,-1.0


In [28]:
top = top_50

rank_data_50 = pd.DataFrame()
    
for year in range(1998, 2024):
    print("Selected Features Top ", top, " Test year:", year)
    
    # Define the range of years for training data
    train_years = range(year - 9, year - 4)  # 5 to 9 years prior to the test year
    
    # Filter the data for training and testing
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'].isin(train_years)]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor', '3P_opponent', '%BLK_nba', '2P%_opponent', 'TOV%.1_misc', 'DRPM_espn', 'WS_advanced', 'war_total_raptor', 'STL_nba', 'MP', '2PA_opponent']  Test year: 1998
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 16.000837234443498
R-squared (XGBoost): -1.5707407015881247


,Player_name,Rank,Pos,Points_won,Predicted_points
13764,Gary Payton,2,PG,37.0,72.997452
13740,Hakeem Olajuwon,-1,C,0.0,72.997452
13894,Dikembe Mutombo,1,C,39.0,72.997452
13605,David Robinson,3,C,10.0,8.847312
13836,Michael Jordan,4,SG,6.0,7.718044
13505,Alonzo Mourning,-1,C,0.0,2.999582
13783,Mario Elie,-1,SF,0.0,2.999582
13626,John Starks,-1,SG,0.0,2.999582
13631,Derrick McKey,-1,SF,0.0,2.999582
13675,Nate McMillan,-1,SG,0.0,2.999582


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor', '3P_opponent', '%BLK_nba', '2P%_opponent', 'TOV%.1_misc', 'DRPM_espn', 'WS_advanced', 'war_total_raptor', 'STL_nba', 'MP', '2PA_opponent']  Test year: 1999
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 12.656257165742554
R-squar

,Player_name,Rank,Pos,Points_won,Predicted_points
13434,David Robinson,4,C,3.0,14.473143
13269,Alonzo Mourning,1,C,89.0,14.171396
13424,Gary Payton,3,PG,6.0,8.686240
13450,Theo Ratliff,5T,PF,2.0,7.228657
13165,Patrick Ewing,-1,C,0.0,6.753525
13370,Shawn Kemp,-1,PF,0.0,6.369448
13182,Dikembe Mutombo,2,C,10.0,6.145478
13488,Tim Duncan,5T,PF,2.0,5.807500
13313,Brevin Knight,-1,PG,0.0,3.753463
13271,Latrell Sprewell,-1,SF,0.0,3.642270


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor', '3P_opponent', '%BLK_nba', '2P%_opponent', 'TOV%.1_misc', 'DRPM_espn', 'WS_advanced', 'war_total_raptor', 'STL_nba', 'MP', '2PA_opponent']  Test year: 2000
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 5.765321235958781
R-square

...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,151.778442
1026,Myles Turner,-1.0,C,0.0,53.400082
813,Ricky Rubio,-1.0,PG,0.0,38.282764
1007,Joel Embiid,-1,C,0.0,25.574015
610,Michael Porter Jr.,-1.0,SF,0.0,24.092148
1046,Sekou Doumbouya,-1.0,PF,0.0,23.800571
762,Jaylen Brown,-1.0,SF,0.0,21.830709
1275,Stephen Curry,-1.0,PG,0.0,20.433477
992,Paul George,-1.0,PF,0.0,20.067423
1123,Draymond Green,10,PF,2.0,19.564564


---------------------------------------------------------------------
Selected Features Top  ['Dpoy_titles', 'war_reg_season_raptor', 'MPG_espn', 'D-LEBRON_bball', 'DWS_advanced', 'predator_defense_raptor', 'AST_opponent', 'STL%_nba', 'FT%_opponent', 'BLK_nba', 'TOV_100_poss', 'pace_impact_raptor', 'BLK_team', 'DBPM_advanced', 'Age', 'TRB_team', 'mp_raptor', 'PER_advanced', 'ORB_opponent', 'W_nba', 'BLK%_advanced', '2P_opponent', 'OPP PTS FB_nba', 'PL_misc', 'Dpoy_votes', 'STL', 'WINS_espn', 'TOV', 'FG%_opponent', 'DEF WS_nba', 'SOS_misc', 'PF_team', 'G', 'WS/48_advanced', 'SRS_misc', 'BLK', 'DEF RTG_nba', 'RPM_espn', 'DRB%_misc', 'poss_raptor', '3P_opponent', '%BLK_nba', '2P%_opponent', 'TOV%.1_misc', 'DRPM_espn', 'WS_advanced', 'war_total_raptor', 'STL_nba', 'MP', '2PA_opponent']  Test year: 2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 341.26446898415566
R-squar

,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,78.006660
76,Mac McClung,-1.0,SG,0.0,70.833649
59,Jaren Jackson Jr.,1.0,C,391.0,18.290918
206,Nic Claxton,9T,C,7.0,10.659180
137,Evan Mobley,3.0,PF,101.0,7.104612
176,Giannis Antetokounmpo,6.0,PF,14.0,5.723259
208,Walker Kessler,-1.0,C,0.0,3.230597
395,Dillon Brooks,-1.0,SF,0.0,2.955334
2,DeMar DeRozan,-1.0,SF,0.0,2.760860
45,Jaden McDaniels,-1.0,SF,0.0,2.636174


---------------------------------------------------------------------


,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,Rank_2004,Rank_2005,Rank_2006,Rank_2007,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,2,4,1,1,1,-1,2,-1,7,-1,...,1.0,4,3,-1.0,15T,2.0,1.0,1.0,3.0,2.0
1,-1,1,7,-1,3T,1,1,1,4,3,...,7.0,2,5.0,-1.0,1.0,1.0,2,9.0,-1.0,-1.0
2,1,3,3T,-1,-1,4,-1,5,4,4,...,13T,1,1,2.0,3,-1.0,10.0,5.0,-1.0,1.0
3,3,5T,-1,2,-1,13T,7,4,13T,24T,...,4.0,-1.0,2,1,6,-1,11,-1,-1,9T
4,4,-1,5T,1,2,13T,-1,-1,-1,13T,...,5,13T,10,-1.0,14.0,5.0,-1,-1.0,-1.0,3.0
5,-1,-1,-1,8T,3T,11,6,9,1,1,...,8T,8,-1,-1,4.0,-1,-1,-1,-1.0,6.0
6,-1,2,8T,3,-1,3,-1.0,-1,10T,6,...,2.0,15T,-1.0,-1,-1.0,-1.0,-1.0,-1,-1.0,-1.0
7,-1,5T,3T,11T,-1,8,10T,18T,6,-1,...,8T,10T,-1.0,5T,-1,3.0,-1.0,-1.0,-1.0,-1.0
8,-1,-1,8T,11T,-1,5.0,-1,8,13T,5,...,13T,-1,4.0,3,-1,-1,5.0,3,-1.0,-1.0
9,-1,-1,-1,5T,-1,6,4,6.0,10T,20.0,...,-1.0,6.0,-1.0,5T,-1,-1,-1.0,-1,10,-1.0


In [29]:
rank_data_10.to_csv('XGB+XGB_Period_rank_data_10.csv', index=False)
rank_data_20.to_csv('XGB+XGB_Period_rank_data_20.csv', index=False)
rank_data_30.to_csv('XGB+XGB_Period_rank_data_30.csv', index=False)
rank_data_40.to_csv('XGB+XGB_Period_rank_data_40.csv', index=False)
rank_data_50.to_csv('XGB+XGB_Period_rank_data_50.csv', index=False)